In [ ]:
#This notebook has the final training of the model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gr
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import holidays

In [2]:
df = pd.read_csv("processed_full.csv")
df.head()

,date,quantity,month,day_of_month,is_month_start,is_month_end,day_of_year,week_of_year,day_of_week,year,...,lag_6,lag_7,rolling_mean_7,rolling_sum_7,rolling_std_7,rolling_mean_30,rolling_sum_30,rolling_std_30,expanding_sum,quantity_winsorized
0,2009-12-01,26434.0,12,1,1,0,335,49,2,2009,...,26434.0,26434.0,23710.571429,165974.0,14932.597372,17233.666667,517010.0,12220.541413,26434.0,26434.0
1,2009-12-02,31938.0,12,2,0,0,336,49,3,2009,...,26434.0,26434.0,23710.571429,165974.0,14932.597372,17233.666667,517010.0,12220.541413,58372.0,31938.0
2,2009-12-03,50898.0,12,3,0,0,337,49,4,2009,...,26434.0,26434.0,23710.571429,165974.0,14932.597372,17233.666667,517010.0,12220.541413,109270.0,50898.0
3,2009-12-04,21405.0,12,4,0,0,338,49,5,2009,...,26434.0,26434.0,23710.571429,165974.0,14932.597372,17233.666667,517010.0,12220.541413,130675.0,21405.0
4,2009-12-05,5119.0,12,5,0,0,339,49,6,2009,...,26434.0,26434.0,23710.571429,165974.0,14932.597372,17233.666667,517010.0,12220.541413,135794.0,5119.0


In [3]:
import itertools
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
import holidays

# 1. Prepare Data
data = df.copy()
data["ds"] = df.index
data["y"] = df["quantity_winsorized"]
data['ds'] = pd.to_datetime(data['ds'])

# Add is_public_holiday column
holiday = holidays.CountryHoliday('UK')
data['is_public_holiday'] = data['ds'].apply(
    lambda date: 1 if date in holiday else 0
)

# Define hyperparameters to tune
param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

best_params = None
best_mape = float('inf')

# Grid search to find the best hyperparameters
for params in all_params:
    model = Prophet(**params)
    model.add_regressor('is_public_holiday')
    model.fit(data)

    # Cross-validate the model
    df_cv = cross_validation(model, initial='547 days', period='180 days', horizon='30 days')
    df_p = performance_metrics(df_cv)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(df_cv['y'], df_cv['yhat'])
    
    if mape < best_mape:
        best_mape = mape
        best_params = params

# Output the best parameters and MAPE
print(f"Best Parameters: {best_params}")
print(f"Best MAPE: {round(best_mape, 2)}")

# Train the final model with the best parameters
model = Prophet(**best_params)
model.add_regressor('is_public_holiday')
model.fit(data)

# Forecast future values
future = model.make_future_dataframe(periods=365)
future['is_public_holiday'] = future['ds'].apply(
    lambda date: 1 if date in holiday else 0
)

forecast = model.predict(future)

# Visualize Results
model.plot(forecast)
model.plot_components(forecast)

# Evaluate Accuracy
df_cv = cross_validation(model, initial='547 days', period='180 days', horizon='30 days')
df_p = performance_metrics(df_cv)
print(df_p.head().round(2))

fig = plot_cross_validation_metric(df_cv, metric='rmse')

# Calculate MAPE using yhat and y
mape = mean_absolute_percentage_error(df_cv['y'], df_cv['yhat'])
print(f"MAPE: {round(mape, 3)}")


15:15:46 - cmdstanpy - INFO - Chain [1] start processing
15:15:47 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Less data than horizon.